In [1]:
%run feature_engineering.ipynb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB
         textID text selected_text sentiment
314  fdb77c3752  NaN           NaN   neutral
<class 'pandas.core.frame.DataFrame'>
Int64Index: 27480 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 

In [28]:
#Creating a list of subsets of words for each tweet and checking which combination gives the highest score based on the dictionary built in feature_engineering
import string
def choose_selected_text(row, tol):
    
    subsets=[]
    
    sentiment=row['sentiment']
    text=row['text']
    
    if sentiment=='neutral':
        return text
    elif sentiment=='positive':
        dict_to_use=pos_dict
    elif sentiment=='negative':
        dict_to_use=neg_dict
    
    words=text.split()
    for i in range(len(words)):
        for j in range(i,len(words)):
            subsets.append(words[i:j+1])

    score=0
    selected_lst=''
    lst=sorted(subsets, key = len)
    
    for i in range(len(subsets)):
        new_sum=0
        for j in range(len(lst[i])):
            #maketrans helps in creating a translate
            if (lst[i][j].translate(str.maketrans('','',string.punctuation)) in dict_to_use.keys()):    #Removes the punctuation from the text by using translate
                new_sum=dict_to_use[lst[i][j].translate(str.maketrans('','',string.punctuation))]       #translate (translate_chars,replace_chars,delete_chars)
                
        if new_sum > score + tol:
            score=new_sum
            selected_lst=lst[i]
            
    if selected_lst=='':
        return text
    
    return selected_lst

In [168]:
#finding out the predicted text for training data
X_train['predicted_text']=''
for index,row in X_train.iterrows():
    
    tol=0.001
    selected_text=choose_selected_text(row,tol)
    X_train.loc[X_train['textID'] == row['textID'], ['predicted_text']] = selected_text

In [169]:
X_train.head()

,textID,text,selected_text,sentiment,predicted_text,predicted_selection,score
21209,306d7b314b,"oh, he`s hilarious. i`m just commenting on th...",hilarious.,positive,hilarious.,hilarious.,1.000000
18265,ce54016d40,"thanks for trying i was hoping bud trillin,...","thanks for trying I was hoping Bud Trillin, ...",positive,thanks,thanks,0.045455
9684,5162aa68ce,after show at our house rocked! saying goodbye...,after show at our house rocked! saying goodbye...,neutral,after show at our house rocked! saying goodbye...,after show at our house rocked! saying goodbye...,1.000000
9689,84bcf86d43,up at 4:30am west coast time..gettin ready to ...,soooo excited to go back to my two fav boyzzzz...,positive,excited,excited,0.083333
22728,ea91e5a7ab,my computer is so slooowww this morning. i th...,My computer is SO slooowww this morning. I th...,neutral,my computer is so slooowww this morning. i th...,my computer is so slooowww this morning. i th...,1.000000


In [170]:
#Validation
X_valid['predicted_text']=''
for index,row in X_valid.iterrows():
    
    tol=0.001
    selected_text=choose_selected_text(row,tol)
    X_valid.loc[X_valid['textID'] == row['textID'], ['predicted_text']] = selected_text

In [171]:
X_valid

,textID,text,selected_text,sentiment,predicted_text,predicted_selection,score
1589,6c5505a37c,enjoy! family trumps everything,Enjoy! Family trumps everything,positive,enjoy!,enjoy!,0.250000
10414,126b1e6a22,--of them kinda turns me off of it all. and ...,kinda turns me off,negative,--of them kinda turns me off of it all. and ...,--of them kinda turns me off of it all. and ...,0.200000
6562,5bc4e623c4,clive it`s my birthday pat me http://apps.fac...,Clive it`s my birthday pat me,neutral,clive it`s my birthday pat me http://apps.fac...,clive it`s my birthday pat me http://apps.fac...,0.857143
2603,984d753104,congrats hey,congrats,positive,congrats,congrats,1.000000
4004,8a79072ca2,is texting,is texting,neutral,is texting,is texting,1.000000
...,...,...,...,...,...,...,...
12296,687aee2f7e,just investigated whether i could change my u...,Just investigated whether I could change my us...,neutral,just investigated whether i could change my u...,just investigated whether i could change my u...,1.000000
617,a82816a060,no one i know likes boiled peanuts t.,no one I know likes boiled peanuts t.,neutral,no one i know likes boiled peanuts t.,no one i know likes boiled peanuts t.,1.000000
4505,f1f588a9bb,awww ) where`d you get that? hugh is so thin....,awww ) where`d you get that? hugh is so thin. ...,neutral,awww ) where`d you get that? hugh is so thin....,awww ) where`d you get that? hugh is so thin....,1.000000
14492,4cca6419e7,im craving some cinnamon toast crunch. why is ...,so cold today!!,negative,im craving some cinnamon toast crunch. why is ...,im craving some cinnamon toast crunch. why is ...,0.250000


In [172]:
def jaccard(actual,pred):
    
    a=set(actual.lower().split())
    b=set(pred.lower().split())
    common=a.intersection(b)
    diff=float(len(common)/(len(a)+len(b)-len(common)))
    return diff

In [173]:
X_valid['score']=X_valid.apply(lambda x: jaccard(x['selected_text'],x['predicted_text']),axis=1)

In [174]:
X_valid

,textID,text,selected_text,sentiment,predicted_text,predicted_selection,score
1589,6c5505a37c,enjoy! family trumps everything,Enjoy! Family trumps everything,positive,enjoy!,enjoy!,0.250000
10414,126b1e6a22,--of them kinda turns me off of it all. and ...,kinda turns me off,negative,--of them kinda turns me off of it all. and ...,--of them kinda turns me off of it all. and ...,0.200000
6562,5bc4e623c4,clive it`s my birthday pat me http://apps.fac...,Clive it`s my birthday pat me,neutral,clive it`s my birthday pat me http://apps.fac...,clive it`s my birthday pat me http://apps.fac...,0.857143
2603,984d753104,congrats hey,congrats,positive,congrats,congrats,1.000000
4004,8a79072ca2,is texting,is texting,neutral,is texting,is texting,1.000000
...,...,...,...,...,...,...,...
12296,687aee2f7e,just investigated whether i could change my u...,Just investigated whether I could change my us...,neutral,just investigated whether i could change my u...,just investigated whether i could change my u...,1.000000
617,a82816a060,no one i know likes boiled peanuts t.,no one I know likes boiled peanuts t.,neutral,no one i know likes boiled peanuts t.,no one i know likes boiled peanuts t.,1.000000
4505,f1f588a9bb,awww ) where`d you get that? hugh is so thin....,awww ) where`d you get that? hugh is so thin. ...,neutral,awww ) where`d you get that? hugh is so thin....,awww ) where`d you get that? hugh is so thin....,1.000000
14492,4cca6419e7,im craving some cinnamon toast crunch. why is ...,so cold today!!,negative,im craving some cinnamon toast crunch. why is ...,im craving some cinnamon toast crunch. why is ...,0.250000


In [175]:
#jaccard score
X_train['score']=X_train.apply(lambda x: jaccard(x['selected_text'],x['predicted_text']),axis=1)

In [176]:
import numpy as np
error_valid=np.mean(X_valid['score'])
error_valid

0.6471797092780285

In [177]:
print(X_train.loc[4,'selected_text'])

Sons of ****,


In [178]:
print(np.mean(X_train['score']))

0.6546512676611831


In [180]:
print(X_train['score'].value_counts())

1.000000    11851
0.000000     2247
0.500000     1122
0.333333      934
0.250000      743
            ...  
0.684211        1
0.521739        1
0.192308        1
0.764706        1
0.107143        1
Name: score, Length: 207, dtype: int64
